In [3]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

# The Usual Suspects

There has been a burglary at a jewelry store.  The usual suspects have been arrested.  These are
<ul>
<li>Aaron,</li>
<li>Bernard, and</li>
<li>Caine.</li>
</ul>
Furthermore, the following facts have been established:
<ol>
<li>It is known that at least one of these suspects is indeed guilty.</li>
<li>If Aaron is guilty, he has exactly one accomplice.</li>
<li>If Bernard is innocent, then Caine is inncocent, too.</li>
<li>If exactly two of the suspects are guilty, then Caine is one of them.</li>
<li>If Caine is innocent, then Aaron is guilty.</li>
</ol>
It is our task to identify those suspects that are guilty.

Our first task is to define the set of propositional variables:
$$ \mathcal{P} := \{ \texttt{a}, \texttt{b}, \texttt{c} \} $$
The interpretation is that 
<ul>
<li>$\texttt{a}$ is true iff Aaron is guilty,</li> 
<li>$\texttt{b}$ is true iff Bernard is guilty, and</li>
<li>$\texttt{c}$ is true iff Caine is guilty.  </li>
</ul>

In [4]:
P = { 'a', 'b', 'c' }

Our next task is to translate the facts given above into formulas from propositional logic. 

The statement "It is known that at least one of these suspects is indeed guilty." is translated as follows:
$$ \texttt{a} \vee \texttt{b} \vee \texttt{c}. $$ 

In [5]:
f1 = 'a ∨ b ∨ c'

The statement "If Aaron is guilty, he has exactly one accomplice." is harder to translate into propositional logic. The idea is to split this statement into two statements:
<ol>
<li>If Aaron is guilty, he has at least one accomplice.</li>
<li>If Aaron is guilty, he has at most  one accomplice.</li>
</ol>
These statements can now be translated into the following formulas:

In [6]:
f2 = 'a → b ∨ c'

In [7]:
f3 = 'a → ¬(b ∧ c)'

The statement "If Bernard is innocent, then Caine is inncocent, too." is a simple implication:

In [8]:
f4 = '¬b → ¬c'

The statement "If exactly two of the suspects are guilty, then Caine is one of them." is best translated into propositional logic by asking how this statement could be made false.
Obviously, this statement is false if two suspects are guilty, but Caine is innocent.
But this is only possible if Caine is innocent and Aaron and Bernard are true.  Hence we can translate this statement as follows:

In [9]:
f5 = '¬(¬c ∧ a ∧ b)'

The statement "If Caine is innocent, then Aaron is guilty." is an implication:

In [10]:
f6 = '¬c → a'

We define the set `Fs` of all formulas:

In [11]:
Fs = { f1, f2, f3, f4, f5, f6 }

We need to transform the strings <tt>f1</tt> to <tt>f6</tt> into nested tuples representing formulas.  To this end we import a parser for propositional formulas.

In [12]:
import propLogParser as plp

def transform(s):
    "transform the string s into a nested tuple"
    return plp.LogicParser(s).parse()

Next, we transform all formulas into nested tuples:

In [19]:
Fs = { transform(f) for f in Fs }

TypeError: expected string or bytes-like object

In [14]:
Fs

{('¬', ('∧', ('∧', ('¬', 'c'), 'a'), 'b')),
 ('→', 'a', ('¬', ('∧', 'b', 'c'))),
 ('→', 'a', ('∨', 'b', 'c')),
 ('→', ('¬', 'b'), ('¬', 'c')),
 ('→', ('¬', 'c'), 'a'),
 ('∨', ('∨', 'a', 'b'), 'c')}

We are looking for a variable assignment $\mathcal{I}$ that satisfies all formulas in the set <tt>Fs</tt>.  As variable assignments are represented as subsets of the set $\mathcal{P}$ of propositional variables, we can just iterate of all subsets of $\mathcal{P}$.

We have previously discussed how to compute the power set $2^M$ of a given set $M$:

In [15]:
def power(M):
    "This function computes the power set of the set M."
    if M == set():
        return { frozenset() }
    else:
        C  = set(M)  # C is a copy of M as we don't want to change the set M
        x  = C.pop() # pop removes some element x from the set C
        P1 = power(C)
        P2 = { A | {x} for A in P1 }
        return P1 | P2

The function $\texttt{evaluate}(F, I)$ takes a propositional formula $F$ and a propositional variable assignment $I$ and evaluates $F$ using the assignment $I$.  We have discussed the details of this function previously.

In [16]:
def evaluate(F, I):
    "Evaluate the propositional formula F using the interpretation I"
    if isinstance(F, str):       # F is a propositional variable
        return F in I            # This variable is true if it occurs in I
    if F[0] == '⊤': return True
    if F[0] == '⊥': return False
    if F[0] == '¬': return not evaluate(F[1], I)
    if F[0] == '∧': return evaluate(F[1], I) and evaluate(F[2], I)
    if F[0] == '∨': return evaluate(F[1], I) or evaluate(F[2], I)
    if F[0] == '→': return not evaluate(F[1], I) or evaluate(F[2], I)
    if F[0] == '↔': return evaluate(F[1], I) == evaluate(F[2], I)

The function `allTrue(Fs, I)` takes a set of propositional formula  `Fs`
and a propositional variable assignment `I`.  It returns `True` only if all formulas from `Fs` are 
`True` given the variable assignment `I`.

In [17]:
def allTrue(Fs, I):
    return all({evaluate(f, I) for f in Fs})

Next, we compute the set of all variable assignments that render all formulas true:

In [18]:
{ I for I in power(P) if allTrue(Fs, I) }

{frozenset({'b', 'c'})}

It turns our that there is just one propositional variable assignment that satisfies all formulas from the set <tt>Fs</tt>.  Therefore, the problem has a unique solution: Bernard and Caine are guilty, while Aaron is innocent.